In [1]:
import pandas as pd
import numpy as np

In [2]:
#pip install transformers


In [3]:
#pip install nltk

In [4]:
#import nltk
#nltk.download()       #punkt select

In [5]:
pip install python 3.n


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement python (from versions: none)
ERROR: No matching distribution found for python


In [6]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import MarianMTModel, MarianTokenizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import re


In [8]:
lemmatizer = WordNetLemmatizer()

In [9]:
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

C:\Users\amrit\Anaconda3\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
def spot_nouns_verbs(en_article):
    words = word_tokenize(en_article)
    tagged_words = pos_tag(words)
    auxiliary_verbs = ['am', 'is', 'are', 'was', 'were', 'has', 'had']
    nouns = [words for words, pos in tagged_words if pos.startswith('NN')]
    # avoids auxiliary verbs
    verbs = [word for word, pos in tagged_words if pos.startswith('VB')and word not in auxiliary_verbs] 
    # to bring verbs to its base form
    verbs = [lemmatizer.lemmatize(verb, pos='v') for verb in verbs]
    # dictionary to store english and HINGLISH nouns
    translated_words = {'feedback':'प्रतिक्रिया', 'definitely':'निश्चित', 'section':'खण्ड'} 
    for noun in nouns:        
            hi_noun = hi_translation(noun)                
            translated_words[noun] = hi_noun    
    for verb in verbs:
        hi_verb = hi_translation(verb)
        modified_value = hi_verb.split(' ', 1)[0]  # Split by the first space and keep the first part
        translated_words[verb] = modified_value    
    return translated_words

In [11]:
# English to Hindi Translation
def hi_translation(en_article):
    inputs = tokenizer.encode(en_article, return_tensors="pt")
    translated_id = model.generate(inputs, max_length=150, num_return_sequences=1, num_beams=4)
    translated_output=tokenizer.decode(translated_id[0], skip_special_tokens=True)  
    translated_output = translated_output.replace('\u200d', '') # handle ZWJ characters
    return translated_output

In [12]:
# Switching Hindi nouns to English nouns to keep certain words in English
def noun_switch(nouns, hinglish_text):
    for key, value in nouns.items():
        matches = re.findall(r'\b' + re.escape(value) + r'\b', hinglish_text)    
        for match in matches:
            hinglish_text = hinglish_text.replace(match, key)
    return hinglish_text

In [14]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amrit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [15]:
# SENTENCE 1
en_article = input("INPUT : ")
nouns = spot_nouns_verbs(en_article)
hi_text = hi_translation(en_article)
# Loop through the dictionary and perform replacements
for eng_word, hin_word in nouns.items():
    hi_text = hi_text.replace(hin_word, eng_word)

print(hi_text)

INPUT : Definitely share your feedback in the comment section.
comment खण्ड में अपनी प्रतिक्रिया को definitely ही share करें ।


In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amrit\AppData\Roaming\nltk_data...


True

In [18]:
# SENTENCE 2
en_article = input("INPUT : ")
nouns = spot_nouns_verbs(en_article)
hi_text = hi_translation(en_article)
# Loop through the dictionary and perform replacements
for eng_word, hin_word in nouns.items():
    hi_text = hi_text.replace(hin_word, eng_word)

print(hi_text)

INPUT : So even if it's a big video, I will clearly mention all the products.
तो यह एक बड़ा video है, तो भी मैं स्पष्ट रूप से सभी productsों का mention करेंगे।


In [19]:
# SENTENCE 3
en_article = input("INPUT : ")
nouns = spot_nouns_verbs(en_article)
hi_text = hi_translation(en_article)

# Loop through the dictionary and perform replacements
for eng_word, hin_word in nouns.items():
    hi_text = hi_text.replace(hin_word, eng_word)

print(hi_text)

INPUT : I was waiting for my bag.
मैं अपने बैग के लिए wait कर रहा था.
